In [ ]:
!pip install transformers gradio scipy ftfy "ipywidgets>=7,<8" datasets accelerate

In [ ]:
!git clone https://github.com/huggingface/diffusers.git

Cloning into 'diffusers'...
remote: Enumerating objects: 64211, done.
remote: Counting objects: 100% (723/723), done.
remote: Compressing objects: 100% (384/384), done.
remote: Total 64211 (delta 459), reused 487 (delta 295), pack-reused 63488
Receiving objects: 100% (64211/64211), 47.89 MiB | 13.51 MiB/s, done.
Resolving deltas: 100% (46960/46960), done.


In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git

  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-ot9qflqc
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-ot9qflqc
  Resolved https://github.com/huggingface/diffusers.git to commit a899e42fc78fbd080452ce88d00dbf704d115280
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.30.0.dev0-py3-none-any.whl size=2209932 sha256=cf56bd87bf237e15c0bb8c539b3cd262eec53268066bd73463cb3f505db494f1
  Stored in directory: /tmp/pip-ephem-wheel-cache-8xypp17_/wheels/4d/b7/a8/6f9549ceec5daad78675b857ac57d697c387062506520a7b50
Successfully built diffusers


In [ ]:
cd diffusers

/content/diffusers


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import gradio as gr
import numpy as np
import random
from diffusers import DiffusionPipeline, StableDiffusionPipeline, UNet2DConditionModel
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
pretrained_model_id =  "CompVis/stable-diffusion-v1-4"
finetuned_model_id = "JuliPolus/sd-1-4-fashion-model"


if torch.cuda.is_available():
    torch.cuda.max_memory_allocated(device=device)

    unet = UNet2DConditionModel.from_pretrained(finetuned_model_id, subfolder="unet_ema", torch_dtype=torch.float16).to(device)
    pipe = StableDiffusionPipeline.from_pretrained(pretrained_model_id,
                                               unet=unet,
                                               torch_dtype=torch.float16,
                                               use_safetensors=True,
                                               variant="fp16",)
    pipe.to(device)
else:

    unet = UNet2DConditionModel.from_pretrained(finetuned_model_id, subfolder="unet_ema").to(device)
    pipe = StableDiffusionPipeline.from_pretrained(pretrained_model_id,
                                               unet=unet,
                                               use_safetensors=True,)

MAX_SEED = np.iinfo(np.int32).max
MAX_IMAGE_SIZE = 512

def infer(prompt, negative_prompt, seed, randomize_seed, width, height, guidance_scale, num_inference_steps):

    if randomize_seed:
        seed = random.randint(0, MAX_SEED)

    generator = torch.Generator().manual_seed(seed)

    image = pipe(
        prompt = prompt,
        negative_prompt = negative_prompt,
        guidance_scale = guidance_scale,
        num_inference_steps = num_inference_steps,
        width = width,
        height = height,
        generator = generator
    ).images[0]

    return image

examples = [
    "bottom, Supreme Animal Print Baggy Jean Washed Indigo, a photography of a dark blue jean with an animal printing on",
    "top, Lacoste Big Elephant Print Sweatshirt Green, a photography of a green sweatshirt with elephant on it",
    "bottom, IAB Studio x Stussy Green palms print Pink Shorts, a photography of Pink pants with a green palms print",
]

css="""
#col-container {
    margin: 0 auto;
    max-width: 520px;
}
"""

if torch.cuda.is_available():
    power_device = "GPU"
else:
    power_device = "CPU"

with gr.Blocks(css=css) as demo:

    with gr.Column(elem_id="col-container"):
        gr.Markdown(f"""
        # Fashion Text-to-Image Generator
        Currently running on {power_device}.
        """)

        with gr.Row():

            prompt = gr.Text(
                value="bottom, Supreme Сhamomile Flower Print Baggy Jean Light Blue, a photography of a Light blue jean with an Сhamomile flower printing on",
                label="Prompt",
                show_label=False,
                max_lines=1,
                placeholder="Enter your prompt",
                container=False,
            )

            run_button = gr.Button("Run", scale=0)

        result = gr.Image(label="Result", show_label=False)

        with gr.Accordion("Advanced Settings", open=False):

            negative_prompt = gr.Text(
                value="Oversaturated, blurry, low quality",
                label="Negative prompt",
                max_lines=1,
                placeholder="Enter a negative prompt",
                visible=True,
            )

            seed = gr.Slider(
                label="Seed",
                minimum=0,
                maximum=MAX_SEED,
                step=1,
                value=500,
            )

            randomize_seed = gr.Checkbox(label="Randomize seed", value=True)

            with gr.Row():

                width = gr.Slider(
                    label="Width",
                    minimum=256,
                    maximum=MAX_IMAGE_SIZE,
                    step=32,
                    value=512,
                )

                height = gr.Slider(
                    label="Height",
                    minimum=256,
                    maximum=MAX_IMAGE_SIZE,
                    step=32,
                    value=512,
                )

            with gr.Row():

                guidance_scale = gr.Slider(
                    label="Guidance scale",
                    minimum=0.0,
                    maximum=10.0,
                    step=0.1,
                    value=9.0,
                )

                num_inference_steps = gr.Slider(
                    label="Number of inference steps",
                    minimum=1,
                    maximum=45,
                    step=1,
                    value=40,
                )

        gr.Examples(
            examples = examples,
            inputs = [prompt]
        )

    run_button.click(
        fn = infer,
        inputs = [prompt, negative_prompt, seed, randomize_seed, width, height, guidance_scale, num_inference_steps],
        outputs = [result]
    )

demo.queue().launch(share=True)

The config attributes {'decay': 0.9999, 'inv_gamma': 1.0, 'min_decay': 0.0, 'optimization_step': 15000, 'power': 0.6666666666666666, 'update_after_step': 0, 'use_ema_warmup': False} were passed to UNet2DConditionModel, but are not expected and will be ignored. Please verify your config.json configuration file.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1aea636918d8ef4349.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
